In [1]:
import pandas as pd
import docx
import redis
import pyarrow as pa

In [2]:
def parse_document(doc_path, skill_path):
    doc = docx.Document(doc_path)
    paras = [x.text for x in doc.paragraphs]
    for i in range(len(paras)):
        if paras[i] == ' ':
            paras[i] = ''
    particular_value = ''
    result = []
    temp_list = []
    for i in paras:
        if i == particular_value:
            temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    df = pd.DataFrame({'text':result})
    tier_list = []
    ability_list = []
    tier = -1
    for _, row in df.iterrows():
        data = row['text']
        if len(data) > 1:
            if data[0].split(' ')[0] == 'Tier':
                tier += 1
                data.pop(0)
            tier_list.append(tier)
            ability_list.append(data)
    ability_df = pd.DataFrame({'Ability':ability_list, 'Tier':tier_list})
    ability_df['Path'] = skill_path
    ability_df['Skill Name'] = ability_df.Ability.apply(lambda x:x[0].split(':')[0])
    ability_df['Description'] = ability_df.Ability.apply(lambda x:x[0].split(':')[1])
    pr = []
    lim = []
    preq = []
    for _ , row in ability_df.iterrows():
        for sublist in row['Ability']:
            if 'Phys Rep' in sublist:
                pr.append(sublist.split(':')[1])
            if 'Limitations' in sublist:
                lim.append(sublist.split(':')[1])
            if 'Prerequisite' in sublist:
                preq.append(sublist.split(':')[1])
        if 'Phys Rep' not in str(row.Ability):
            pr.append(None)
        if 'Limitations' not in str(row.Ability):
            lim.append(None)
        if 'Prerequisite' not in str(row.Ability):
            preq.append(None)
    ability_df['Phys Rep'] = pr
    ability_df['Limitations'] = lim
    ability_df['Prerequisite'] = preq
    ability_df = ability_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return ability_df[['Skill Name', 'Description', 'Path', 'Tier', 'Limitations', 'Phys Rep', 'Prerequisite']]

In [3]:
warrior_df = parse_document("The Warrior's Path.docx", 'Warrior')
rogue_df = parse_document("The Rogue's Path.docx", 'Rogue')
healer_df = parse_document("The Healer's Path.docx", 'Healer')
mage_df = parse_document("The Mage's Path.docx", 'Mage')
bard_df = parse_document("The Bard's Path.docx", 'Bard')

In [4]:
skills = pd.concat([warrior_df,rogue_df,healer_df,mage_df,bard_df])

In [5]:
skills

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,None,None,None
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,None
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,None,None,None
4,Advanced Weapon Training,Study and become proficient in the use of two-...,Warrior,1,None,None,None
...,...,...,...,...,...,...,...
8,Curative Works,The Bard is able to channel more powerful heal...,Bard,3,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played for specifi...",Healing Song
9,Song of Sight,The Bard is able to play a melody that resonat...,Bard,3,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played towards the...",None
10,Group Inspiration,The Bard greatly projects their voice and thei...,Bard,4,The additional armor point only lasts only as ...,A 60 second song intro is played to a group of...,Party Inspiration
11,Song of Life,The Bard can fully restore a mortally wounded ...,Bard,5,Can only be used once a day plus one for every...,"A song of at least 2 minutes, played to a mort...",Curative Works


In [6]:
skills.to_excel('Skills_Table.xlsx', index=False)

In [7]:
skills.to_csv('Skills_Table.csv', index=False)

In [8]:
creds_provider = redis.UsernamePasswordCredentialProvider("default", "GyDBQLERll5EBie5jJ7TXdcn8qe8exfc")
user_connection = redis.Redis(host="redis-15391.c232.us-east-1-2.ec2.redns.redis-cloud.com", port=15391, credential_provider=creds_provider)

In [9]:
user_connection.set("skills", pa.serialize_pandas(skills).to_pybytes())

True

In [10]:
pa.deserialize_pandas(user_connection.get("skills"))

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,None,None,None
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,None
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,None,None,None
4,Advanced Weapon Training,Study and become proficient in the use of two-...,Warrior,1,None,None,None
...,...,...,...,...,...,...,...
8,Curative Works,The Bard is able to channel more powerful heal...,Bard,3,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played for specifi...",Healing Song
9,Song of Sight,The Bard is able to play a melody that resonat...,Bard,3,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played towards the...",None
10,Group Inspiration,The Bard greatly projects their voice and thei...,Bard,4,The additional armor point only lasts only as ...,A 60 second song intro is played to a group of...,Party Inspiration
11,Song of Life,The Bard can fully restore a mortally wounded ...,Bard,5,Can only be used once a day plus one for every...,"A song of at least 2 minutes, played to a mort...",Curative Works
